# Step 1

One of the main differences between the Heston model and the Black-Scholes model lies in their treatment of volatility. In the Black-Scholes model, volatility is assumed to be constant over the life of the option. 

In contrast, the Heston model allows for stochastic volatility, meaning that volatility itself is modeled as a random process. It assumes that the underlying asset's volatility follows a square-root diffusion process, known as a stochastic volatility process. This allows the model to capture the observed volatility clustering and mean-reversion properties commonly seen in financial markets.

The stock price satisfies: 

$dS_t = \mu S_t\,dt + \sqrt{\nu_t} S_t\,dW^S_t$

and the volatility satisfies

$d\nu_t = \kappa(\theta - \nu_t)\,dt + \sigma \sqrt{\nu_t}\,dW^{\nu}_t$

Here:

* $\nu_0$ is the initial variance
* $\kappa$ is the rate at which $\nu$ reverts to the mean
* $\theta$ is the long-run variance
* $W^S_t$ and $W^\nu_t$ are Wiener processes whose correlation is $\rho$
* $\sigma$ is the volatility of volatility

We compute option prices in the Heston Model using the following parameters

* $S_0 = 80$
* $r = 5.5\%$
* $\sigma = 35\%$
* Time to maturity = $3$ months.


For the Heston model, we use the following parameters

* $\nu_0 = 3.2\%$
* $\kappa_\nu = 1.85$
* $\theta_\nu = 0.045$

## Question 5

We use Monte-Carlo simulation in a Heston model to price At-the-Money European call and put options.  We use a correlation value of $-0.30$.

To do this, we create a function `generate_heston_paths' that uses the NumPy library to generate paths for the Heston stochastic volatility model. It iterates over a specified number of steps to simulate asset price and volatility paths based on the Heston model dynamics. The function returns an array of asset price paths, and optionally, an array of volatility paths if specified. By leveraging NumPy's vectorized computations, the code efficiently generates paths for the Heston model, making it suitable for financial applications involving stochastic volatility analysis.

In [27]:
import numpy as np
def generate_heston_paths(S, T, r, kappa, theta, v_0, rho, sigma, 
                          steps, Npaths, return_vol=False):
    dt = T/steps
    size = (Npaths, steps)
    prices = np.zeros(size)
    sigs = np.zeros(size)
    S_t = S
    v_t = v_0
    for t in range(steps):
        WT = np.random.multivariate_normal(np.array([0,0]), 
                                           cov = np.array([[1,rho],
                                                          [rho,1]]), 
                                           size=Npaths) * np.sqrt(dt) 
        
        S_t = S_t*(np.exp( (r- 0.5*v_t)*dt+ np.sqrt(v_t) *WT[:,0] ) ) 
        v_t = np.abs(v_t + kappa*(theta-v_t)*dt + sigma*np.sqrt(v_t)*WT[:,1])
        prices[:, t] = S_t
        sigs[:, t] = v_t
    
    if return_vol:
        return prices, sigs
    
    return prices

In [ ]:
def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand

In [29]:
v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.30

S0 = 80 # Current underlying asset price
r = 0.055  # Risk-free rate
T = 3/12 # Number of years
M = 180  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

In [ ]:
np.random.seed(42)
# Generating random numbers from standard normal

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

To calculate the price of a put or call option in the Heston model, we compute the option payoffs, discount the expected payoff to the present time at the risk-free rate of interest.

In [ ]:
def heston_call_mc(S0, K, r, T, t):
    np.random.seed(42)
    S = generate_heston_paths(S0, T, r, kappa_v, theta_v, v0, rho, sigma_v, M, Ite)
    payoff = np.maximum(0, -K + S[:, -1])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [ ]:
def heston_put_mc(S0, K, r, T, t):
    np.random.seed(42)
    S = generate_heston_paths(S0, T, r, kappa_v, theta_v, v0, rho, sigma_v, M, Ite)
    payoff = np.maximum(0, K - S[:, -1])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [ ]:
print(f"European Call Price under Heston: {heston_call_mc(80, 80, 0.055, 3/12, 0):.2f}")
print(f"European Put Price under Heston: {heston_put_mc(80, 80, 0.055, 3/12, 0):.2f}")

European Call Price under Heston: 3.46
European Put Price under Heston: 2.37


In [ ]:
np.random.seed(42)
# Covariance Matrix
rho = -0.70
covariance_matrix = np.zeros((2, 2), dtype=np.float64)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

## Question 6

In [ ]:
print(f"European Call Price under Heston: {heston_call_mc(80, 80, 0.055, 3/12, 0):.2f}")
print(f"European Put Price under Heston: {heston_put_mc(80, 80, 0.055, 3/12, 0):.2f}")

European Call Price under Heston: 3.49
European Put Price under Heston: 2.40


correlation value |Call Price | Put Price
---|---|---
$\rho = -0.30$|3.46|2.37
$\rho = -0.70$|3.49|2.40


## Question 7

Delta represents the sensitivity of an option's price to changes in the underlying asset's price. To compute delta, we use the `heston_call_mc` and `heston_put_mc` function to estimate the option prices for two scenarios: the original asset price and a slightly increased asset price. By computing the difference in prices and dividing it by the small change in asset price, the function approximates the delta of the call option. The resulting delta value indicates the expected change in the option price for a unit change in the underlying asset price.

In [ ]:
def heston_call_delta(S, K, r, T, t):
    price1 = heston_call_mc(S, K, r, T, t)
    price2 = heston_call_mc(S+0.01, K, r, T, t)
    return (price2 - price1) / 0.01

In [ ]:
def heston_put_delta(S, K, r, T, t):
    price1 = heston_put_mc(S, K, r, T, t)
    price2 = heston_put_mc(S+0.01, K, r, T, t)
    return (price2 - price1) / 0.01

In [ ]:
rho = -0.30
print(f"European Call Delta under Heston: {heston_call_delta(80, 80, 0.055, 3/12, 0):.3f}")

European Call Delta under Heston: 0.604


In [ ]:
print(f"European Put Delta under Heston: {heston_put_delta(80, 80, 0.055, 3/12, 0):.3f}")

European Put Delta under Heston: -0.396


 Gamma measures the rate of change of an option's delta with respect to changes in the underlying asset's price.  We calculate gamma by computing the second-order difference in prices and dividing it by the squared small change in asset price, the function approximates the gamma of the call option. 

In [ ]:
def heston_call_gamma(S, K, r, T, t):
    a = heston_call_mc(S+0.01, K, r, T, t)
    b = heston_call_mc(S, K, r, T, t)
    c = heston_call_mc(S-0.01, K, r, T, t)
    return (a-2*b+c) / (0.01**2)

In [ ]:
def heston_put_gamma(S, K, r, T, t):
    a = heston_put_mc(S+0.01, K, r, T, t)
    b = heston_put_mc(S, K, r, T, t)
    c = heston_put_mc(S-0.01, K, r, T, t)
    return (a-2*b+c) / (0.01**2)

In [ ]:
print(f"European Call Gamma under Heston: {heston_call_gamma(80, 80, 0.055, 3/12, 0):.3f}")
print(f"European Put Gamma under Heston: {heston_put_gamma(80, 80, 0.055, 3/12, 0):.3f}")


European Call Gamma under Heston: 0.060
European Put Gamma under Heston: 0.060


In [ ]:
np.random.seed(0)
# Covariance Matrix
rho = -0.70
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

<ipython-input-203-0f2eaf2d0669>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)


In [ ]:
print(f"European Call Delta under Heston: {heston_call_delta(80, 80, 0.055, 1, 0):.3f}")
print(f"European Put Delta under Heston: {heston_put_delta(80, 80, 0.055, 1, 0):.3f}")

European Call Delta under Heston: 0.325
European Put Delta under Heston: -0.577


In [ ]:
print(f"European Call Gamma under Heston: {heston_call_gamma(80, 80, 0.055, 1, 0):.3f}")
print(f"European Put Gamma under Heston: {heston_put_gamma(80, 80, 0.055, 1, 0):.3f}")

European Call Gamma under Heston: 0.055
European Put Gamma under Heston: 0.055



We price options using the Merton model.  We use the following parameters

* $\mu = -0.5$
* $\delta = 0.22$


In [9]:
mu = -0.5
delta = 0.22

## Question 8

The Merton model extends the Black-Scholes model by allowing the stock price to have jumps. It includes both a Brownian motion for diffusion as well as a Poisson jump process.  The frequency of the jump is given by a Poisson processs with intensity $\lambda$.  The size of the jumps has the distribution $Log(1 + J_t ) ≈ \rm{Normal} (\log (1 + \mu_j ) − δ^2/2, \delta^2)$. The jumps represent sudden and significant changes in the asset's value, which can be caused by unexpected news, events, or market shocks.

The model assumes that the asset price follows a geometric Brownian motion during the periods between jumps, similar to the Black-Scholes framework. The diffusion component accounts for the continuous and smooth movement of the asset price, reflecting the impact of random market fluctuations.



In [6]:
lamb = 0.75
r = 0.055  # Risk-free rate
sigma = 0.35  # Volatility
T = 3/12  # Maturity/time period (in years)
S0 = 80  # Current Stock Price

Ite = 100000  # Number of simulations (paths)
M = 540  # Number of steps
dt = T / M  # Time-step

In [ ]:
np.random.seed(42)
SM = np.zeros((M + 1, Ite))
SM[0] = S0

rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))

In [ ]:
for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!

In [12]:
def merton_call_mc(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [ ]:
print(f"European Call Price under Merton: {merton_call_mc(SM, 80, r, T, 0):.2f}")

European Call Price under Merton: 8.24


In [13]:
def merton_put_mc(S, K, r, T, t):
    payoff = np.maximum(0, K - S[-1, :])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [ ]:
print(f"European Put Price under Merton: {merton_put_mc(SM, 80, r, T, 0):.2f}")

European Put Price under Merton: 7.23


## Question 9

In [14]:
lamb = 0.25
r = 0.055  # Risk-free rate
sigma = 0.35  # Volatility
T = 3/12  # Maturity/time period (in years)
S0 = 80  # Current Stock Price

Ite = 100000  # Number of simulations (paths)
M = 540  # Number of steps
dt = T / M  # Time-step
np.random.seed(42)
SM = np.zeros((M + 1, Ite))
SM[0] = S0

rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))
for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!

In [15]:
print(f"European Call Price under Merton: {merton_call_mc(SM, 80, r, T, 0):.2f}")

European Call Price under Merton: 6.77


In [16]:
print(f"European Put Price under Merton: {merton_put_mc(SM, 80, r, T, 0):.2f}")

European Put Price under Merton: 5.75


This table shows call and put prices for an ATM option with strike price $80$ dated 3 months in the future.  As in the previous question the risk-free rate is 5.5 per cent per annum, and the volatility is 35 per cent per annum.

$\lambda$|call price|put price
---|---|---
0.75|8.24|7.23
0.25|6.77|5.75


## Question 10

In [17]:
import numpy as np
def merton_path(S0):
  np.random.seed(42)
  SM = np.zeros((M + 1, Ite))
  SM[0] = S0

  rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

  # Random numbers
  z1 = np.random.standard_normal((M + 1, Ite))
  z2 = np.random.standard_normal((M + 1, Ite))
  y = np.random.poisson(lamb * dt, (M + 1, Ite))
  for t in range(1, M + 1):
      SM[t] = SM[t - 1] * (
          np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
          + (np.exp(mu + delta * z2[t]) - 1) * y[t]
      )
      SM[t] = np.maximum(
          SM[t], 0.00001
      )  # To ensure that the price never goes below zero!
  return SM

In [18]:
def merton_call_delta(S0, K, r, T, t):
  a = merton_path(S0)
  price1 = merton_call_mc(a, K, r, T, t)
  b = merton_path(S0+0.01)
  price2 = merton_call_mc(b, K, r, T, t)
  return (price2 - price1)/0.01

def merton_put_delta(S0, K, r, T, t):
  a = merton_path(S0)
  price1 = merton_put_mc(a, K, r, T, t)
  b = merton_path(S0+0.01)
  price2 = merton_put_mc(b, K, r, T, t)
  return (price2 - price1)/0.01

In [19]:
lamb=0.75
print(f"Delta for the call at lambda=0.75 is {merton_call_delta(80, 80, 0.055, T, 0):.2f}")
print(f"Delta for the put at lambda=0.75 is {merton_put_delta(80, 80, 0.055, T, 0):.2f}")

Delta for the call at lambda=0.75 is 0.65
Delta for the put at lambda=0.75 is -0.35


In [20]:
lamb=0.25
print(f"Delta for the call at lambda=0.25 is {merton_call_delta(80, 80, 0.055, T, 0):.2f}")
print(f"Delta for the put at lambda=0.25 is {merton_put_delta(80, 80, 0.055, T, 0):.2f}")

Delta for the call at lambda=0.25 is 0.60
Delta for the put at lambda=0.25 is -0.40


In [21]:
def merton_call_gamma(S0, K, r, T, t):
  a = merton_path(S0)
  price1 = merton_call_mc(a, K, r, T, t)
  b = merton_path(S0+0.01)
  price2 = merton_call_mc(b, K, r, T, t)
  c = merton_path(S0-0.01)
  price3 = merton_call_mc(c, K, r, T, t)
  return (price2 - 2*price1 + price3)/0.01**2

def merton_put_gamma(S0, K, r, T, t):
  a = merton_path(S0)
  price1 = merton_put_mc(a, K, r, T, t)
  b = merton_path(S0+0.01)
  price2 = merton_put_mc(b, K, r, T, t)
  c = merton_path(S0-0.01)
  price3 = merton_put_mc(c, K, r, T, t)
  return (price2 - 2*price1 + price3)/0.01**2

In [22]:
lamb=0.75
print(f"Gamma for the call at lambda=0.75 is {merton_call_gamma(80, 80, 0.055, T, 0):.4f}")
print(f"Gamma for the put at lambda=0.75 is {merton_put_gamma(80, 80, 0.055, T, 0):.4f}")

Gamma for the call at lambda=0.75 is 0.0300
Gamma for the put at lambda=0.75 is 0.0300


In [23]:
lamb=0.25
print(f"Gamma for the call at lambda=0.25 is {merton_call_gamma(80, 80, 0.055, T, 0):.4f}")
print(f"Gamma for the put at lambda=0.25 is {merton_put_gamma(80, 80, 0.055, T, 0):.4f}")

Gamma for the call at lambda=0.25 is 0.0240
Gamma for the put at lambda=0.25 is 0.0240


## Question 11



For $\rho = -0.30$:

In [ ]:
bond_price = S0 * np.exp(-r * T)
print(f"The price of the zero-coupon bond at 5.5% interest rate is: {bond_price:.2f}")

The price of the zero-coupon bond at 5.5% interest rate is: 78.91


In [ ]:
print(f"Call price plus Bond Price = {bond_price + 3.46:.2f}")
print(f"Put price plus Stock Price = {S0 + 2.37:.2f}")

Call price plus Bond Price = 82.37
Put price plus Stock Price = 82.37


For $\rho = -0.70$:

In [ ]:
print(f"Call price plus Bond Price = {bond_price + 3.49:.2f}")
print(f"Put price plus Stock Price = {S0 + 2.40:.2f}")

Call price plus Bond Price = 82.40
Put price plus Stock Price = 82.40


At either the $\rho = -0.30$ or $\rho = -0.70$ correlation level we have put-call parity.

For $\lambda = 0.75$:

In [ ]:
print(f"Bond price plus call price is {bond_price + 8.24:.2f}")

Bond price plus call price is 87.21


In [ ]:
print(f"Stock price plus put price is {80 + 7.23:.2f}")

Stock price plus put price is 87.23


For $\lambda = 0.25$


In [ ]:
print(f"Bond price plus call price is {bond_price + 6.77:.2f}")

Bond price plus call price is 85.68


In [ ]:
print(f"Stock price plus put price is {80 + 5.74:.2f}")

Stock price plus put price is 85.74


For the Merton model at either $\lambda = 0.75$ or $\lambda = 0.25$ we have put call parity (within 10 cents)

## Question 12


For the Heston model at $\rho = -0.30$:

In [ ]:
moneyness = [0.85, 0.90, 0.95, 1, 1.05, 1.10, 1.15]
strikes = [S0 * m for m in moneyness]

In [ ]:
np.random.seed(42)
# Generating random numbers from standard normal

# Covariance Matrix
rho = -0.30
covariance_matrix = np.zeros((2, 2), dtype=float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

In [ ]:
heston_prices = [heston_call_mc(S0, K, 0.055, T, 0) for K in strikes]
print ("Strike\tCall Price")
print("--------------------")
for (strike, price) in zip(strikes, heston_prices):
  print(f"{strike}\t{price:.2f}")

Strike	Call Price
--------------------
68.0	13.06
72.0	9.40
76.0	6.10
80	3.46
84.0	1.66
88.0	0.68
92.0	0.25


For the Heston model at $\rho = -0.70$

In [ ]:
np.random.seed(42)
# Generating random numbers from standard normal

# Covariance Matrix
rho = -0.70
covariance_matrix = np.zeros((2, 2), dtype=float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)
heston_prices2 = [heston_call_mc(S0, K, 0.055, T, 0) for K in strikes]
print ("Strike\tCall Price")
print("--------------------")
for (strike, price) in zip(strikes, heston_prices2):
  print(f"{strike}\t{price:.2f}")

Strike	Call Price
--------------------
68.0	13.13
72.0	9.50
76.0	6.21
80	3.49
84.0	1.56
88.0	0.51
92.0	0.11


For the Merton model at $\lambda = 0.75$:

In [ ]:
lamb = 0.75
np.random.seed(42)
SM = np.zeros((M + 1, Ite))
SM[0] = S0

rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))
for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!
merton_prices = [merton_call_mc(SM, K, 0.0555, T, 0) for K in strikes]

In [ ]:
print ("Strike\tCall Price")
print("--------------------")
for (strike, price) in zip(strikes, merton_prices):
  print(f"{strike}\t{price:.2f}")

Strike	Call Price
--------------------
68.0	16.22
72.0	13.26
76.0	10.59
80	8.24
84.0	6.25
88.0	4.62
92.0	3.33


For the Merton model at $\lambda = 0.25$:

In [ ]:
lamb = 0.25
np.random.seed(42)
SM = np.zeros((M + 1, Ite))
SM[0] = S0

rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))
for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001
    )  # To ensure that the price never goes below zero!
merton_prices = [merton_call_mc(SM, K, 0.0555, T, 0) for K in strikes]

In [ ]:
print ("Strike\tCall Price")
print("--------------------")
for (strike, price) in zip(strikes, merton_prices):
  print(f"{strike}\t{price:.2f}")

Strike	Call Price
--------------------
68.0	14.71
72.0	11.68
76.0	9.02
80	6.76
84.0	4.93
88.0	3.50
92.0	2.42


# Step 2

## Question 13

The `heston_call_mc_american` and `heston_put_mc_american` functions use Monte Carlo simulation and backward induction to price American-style call and put options within the Heston stochastic volatility model. We generate Heston paths for the asset price, iterate through each time step in reverse order, estimate the conditional expectation of future cashflows using polynomial regression, compare the intrinsic value of the option with the continuation value to determine exercise decisions, and update the data accordingly. The function calculates the average value of the remaining data as the expected payoff of the American call or put option, discounts it to the present time, and returns the estimated price.

In [24]:
import warnings
warnings.filterwarnings("ignore")
def heston_call_mc_american(S0, K, r, T):
    np.random.seed(42)
    S = generate_heston_paths(S0, T, r, kappa_v, theta_v, v0, rho, sigma_v, M, Ite)
    data = np.maximum(0, -K + S[:,-1])
    for t in range(M - 2, -1, -1):
        X = S[:,t]
        intrinsic = np.maximum(X - K, 0)
        Y = np.exp(-r * dt) * data

        # Polynomial regression to estimate the conditional expectation
        cashflows = np.polyfit(X, Y, 5)
        continuation = np.polyval(cashflows, X)

        # Compare the intrinsic value with the continuation value
        exercise = intrinsic > continuation
        data = np.where(exercise, intrinsic, data)

    average = np.sum(data) / float(Ite)

    return np.exp(-r * T) * average

In [25]:
import numpy as np
rho = -0.30
covariance_matrix = np.zeros((2, 2), dtype=float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

In [30]:
print(f"The price of an American call option using Monte-Carlo is {heston_call_mc_american(80, 80, 0.055, 3/12):.2f}")

The price of an American call option using Monte-Carlo is 3.24


In [31]:
def heston_put_mc_american(S0, K, r, T):
    np.random.seed(42)
    S = generate_heston_paths(S0, T, r, kappa_v, theta_v, v0, rho, sigma_v, M, Ite)
    data = np.maximum(0, K - S[:,-1])
    for t in range(M - 2, -1, -1):
        X = S[:,t]
        intrinsic = np.maximum(K-X, 0)
        Y = np.exp(-r * dt) * data

        # Polynomial regression to estimate the conditional expectation
        cashflows = np.polyfit(X, Y, 5)
        continuation = np.polyval(cashflows, X)

        # Compare the intrinsic value with the continuation value
        exercise = intrinsic > continuation
        data = np.where(exercise, intrinsic, data)

    average = np.sum(data) / float(Ite)

    return np.exp(-r * T) * average

In [32]:
print(f"The price of an American put option using Monte-Carlo is {heston_put_mc_american(80, 80, 0.055, 3/12):.2f}")

The price of an American put option using Monte-Carlo is 2.29


Let's compute it in the Merton model with $\lambda = 0.75$:

In [33]:
SM = merton_path(80)
K=80
data = np.maximum(0, K - SM[-1, :])
for t in range(M - 2, -1, -1):
    X = SM[t, :]
    intrinsic = np.maximum(K-X, 0)
    Y = np.exp(-r * dt) * data

    # Polynomial regression to estimate the conditional expectation
    cashflows = np.polyfit(X, Y, 5)
    continuation = np.polyval(cashflows, X)

    # Compare the intrinsic value with the continuation value
    exercise = intrinsic > continuation
    data = np.where(exercise, intrinsic, data)

    average = np.sum(data) / float(Ite)
amer_put_price = np.exp(-r * T) * average

K=80
data = np.maximum(0, -K + SM[-1, :])
for t in range(M - 2, -1, -1):
    X = SM[t, :]
    intrinsic = np.maximum(-K+X, 0)
    Y = np.exp(-r * dt) * data

    # Polynomial regression to estimate the conditional expectation
    cashflows = np.polyfit(X, Y, 5)
    continuation = np.polyval(cashflows, X)

    # Compare the intrinsic value with the continuation value
    exercise = intrinsic > continuation
    data = np.where(exercise, intrinsic, data)

    average = np.sum(data) / float(Ite)

amer_call_price = np.exp(-r * T) * average

In [34]:
print(f"The price of an American call option is {amer_call_price:.2f}")
print(f"The price of an American put option is {amer_put_price:.2f}")

The price of an American call option is 6.37
The price of an American put option is 5.50


Now let's compute it with $\lambda = 0.25$:


In [35]:
lamb = 0.25
SM = merton_path(80)
K=80
data = np.maximum(0, K - SM[-1, :])
for t in range(M - 2, -1, -1):
    X = SM[t, :]
    intrinsic = np.maximum(K-X, 0)
    Y = np.exp(-r * dt) * data

    # Polynomial regression to estimate the conditional expectation
    cashflows = np.polyfit(X, Y, 5)
    continuation = np.polyval(cashflows, X)

    # Compare the intrinsic value with the continuation value
    exercise = intrinsic > continuation
    data = np.where(exercise, intrinsic, data)

    average = np.sum(data) / float(Ite)
amer_put_price = np.exp(-r * T) * average

K=80
data = np.maximum(0, -K + SM[-1, :])
for t in range(M - 2, -1, -1):
    X = SM[t, :]
    intrinsic = np.maximum(-K+X, 0)
    Y = np.exp(-r * dt) * data

    # Polynomial regression to estimate the conditional expectation
    cashflows = np.polyfit(X, Y, 5)
    continuation = np.polyval(cashflows, X)

    # Compare the intrinsic value with the continuation value
    exercise = intrinsic > continuation
    data = np.where(exercise, intrinsic, data)

    average = np.sum(data) / float(Ite)

amer_call_price = np.exp(-r * T) * average

In [36]:
print(f"The price of an American call option is {amer_call_price:.2f}")
print(f"The price of an American put option is {amer_put_price:.2f}")

The price of an American call option is 6.37
The price of an American put option is 5.50


## Question 14

In [ ]:
def heston_call_mc_barrier(S0, K, r, T, barrier):
    np.random.seed(42)
    S = generate_heston_paths(S0, T, r, kappa_v, theta_v, v0, rho, sigma_v, M, Ite)
    data = np.maximum(0, -K + S[:,-1])
    crossed_barrier = np.any(S[:,:-1] >= barrier, axis=1)
    
    # Calculate the payoff for each path
    data = np.where(crossed_barrier, S[:,-1] - K, 0)
    
    average = np.sum(np.maximum(data, 0)) / float(Ite)

    return np.exp(-r * T) * average

In [ ]:
print(f"The price of the up and in barrier option is {heston_call_mc_barrier(80, 95, 0.055, 3/12, 95):.2f}")
print(f"The price of the vanilla call option is {heston_call_mc(80, 95, 0.055, 3/12, 0):.2f}")

The price of the up and in barrier option is 0.11
The price of the vanilla call option is 0.11


The prices are the same because the barrier level is the same as the strike price.  Hence if the option ends in the money, it goes above the barrier. In this scenario, if the underlying asset price reaches or exceeds the barrier before the option's expiration, the option is knocked in and becomes active. Since the barrier and strike prices are identical, the option is already active, and its payout is determined solely by the asset price at expiration, just like a vanilla option. Therefore, the price of an up-and-in barrier option with barrier price equal to the strike price is equivalent to the price of a vanilla option with the same strike price and other parameters.

## Question 15

In [ ]:
lamb = 0.75
S = merton_path(80)
K = 65
crossed_barrier = np.any(S[:-1, :] <= 65, axis=0)
data = np.where(crossed_barrier, K - S[-1,:], 0)
average = np.sum(np.maximum(data, 0)) / float(Ite)
barrier_put_price = np.exp(-r * T) * average

In [ ]:
print(f"The price of the down and in put option is {barrier_put_price:.2f}")

The price of the down and in put option is 2.77


In [ ]:
print(f"The price of the vanilla option is {merton_put_mc(S, 65, 0.055, 3/12, 0):.2f}")

The price of the vanilla option is 2.77


The price of the vanilla put and the down and in put are the same because the option becomes alive if the stock price reaches the level.  But if the option ends up in the money, the stock must have reached the barrier level. When the barrier price and strike price of a down-and-in put barrier option are the same, the option behaves like a standard vanilla put option. In this case, if the underlying asset price falls below or touches the barrier before the option's expiration, the option is knocked in and becomes active. Since the barrier and strike prices are equal, the option is already active, and its payout is determined solely by the asset price at expiration, resembling a vanilla put option. Hence, the price of a down-and-in put barrier option with a barrier price equal to the strike price is equivalent to the price of a vanilla put option with the same strike price and other relevant parameters.